## k-degree sequence Anonymisation

I was initially hoping to use the java implemntaition byt that straight up does not work for the HBA graphs I tried it on.

The algorithm works by first grouping the degree sequence into groups with at least k elements. Each group is given a target degree, and the edges of the graph are modified to fit the target degree sequence.

Uses the optimal univariate micro-aggrefation to achieve best group distribution. Then compute values for each group that minimize delta from original deg seq. delta = sum from  i=1 to n of |d_i tilde - d_i|.

compute the groups g = {g1,...,gp} n/k<=p<=2/(2k-1) each group between k and 2k-1 items
compute M (p x 2) first col is sum of differences between each element of the group and the arithmetic mean of all degrees that pelong to the group using floor funtion to round the mean value. Second column is the same but the ceiling function is used. 
mj1 = sum over di in gj of (di - floor(<gj>)) where <gj> is the avg value of di in gj.

In [13]:
import networkx as nx
import numpy as np

In [10]:
from microagg1d import univariate_microaggregation

In [23]:
DD199 = nx.read_edgelist("Data/DD199/DD199.edges", nodetype=int)

BAG = nx.barabasi_albert_graph(20, 2)

In [39]:
def greedyDegSeqAnon(m, probs, maxIt = 20):
    '''Values for mj are selected acc to prob distribution based on size of mj1 and mj2.
    process is finished when a solution is found s.t. sum j=1 to p of mj = 0
    or when we go a certain number of iterations without change.'''
    ## selects based on probabilities
    ## saves best solution (w.r.t. the sum of all els)
    ## if no change within 20 iters, stop

    ## just get some option to start
    bestM = {j: m[j][0] if random.random() <= probs[j][0] else m[j][1] for j in m}
    sumM = abs(sum(bestM.values()))

    ## select new option
    daysSinceLastChange = 0
    while daysSinceLastChange < maxIt:
        newM = {j: m[j][0] if random.random() <= probs[j][0] else m[j][1] for j in m}
        if abs(sum(newM.values())) < sumM:
            bestM = newM
            sumM = abs(sum(newM.values()))
            daysSinceLastChange = 0
        else:
            daysSinceLastChange += 1
    return bestM

In [14]:
import random

In [1]:
def k_degSeq_Anon(G:nx.Graph(), k= 10):    
    ## clustering the degree sequence
    d = sorted(dict(G.degree()).items(), key = lambda x: x[1], reverse = True)
    nodesL = [n for (n,_) in d]
    degL = [deg for (_,deg) in d]
    clusters = univariate_microaggregation(degL, k=k)
    ## list of cluster numbers, ordered as in nodesL

    ## want to create mj1 and mj2 for each group j
    ## need the avg deg of each cluster
    ## create dictionary s.t. key is cluster number, and value is list of corresponding nodes
    clusters_degsDict = {j:[] for j in set(clusters)} ## get only the cluster numbers
    for i in range(len(degL)):
        clusters_degsDict[clusters[i]].append(degL[i])
    ## now we have clustersDict as {cluster number: [di,...,dn]} for i,...,n in cluster
    sumd = {j:(sum(clusters_degsDict[j]), len(clusters_degsDict[j])) for j in clusters_degsDict}
    m = {j:(sumd[j][0]-sumd[j][1]*np.floor(sumd[j][0] / sumd[j][1]), sumd[j][0]-sumd[j][1]*np.ceil(sumd[j][0] / sumd[j][1])) for j in sumd}
    probs = {j:(1- m[j][0]/(m[j][0]-m[j][1]),1 + m[j][1]/(m[j][0]-m[j][1])) for j in m}
    ## use implied def for probs rather than stated (stated def not in range[0,1])
    ## sometimes encounters div 0 error, but that seems fine, since => mj1=mj2=0, so no change necc
    
    bestM = greedyDegSeqAnon(m, probs)
    ## bestM is a dictionary {j: mj1 or mj2}
    ## we know which one based on the sign, if (-) then mj2
    targets = {j: np.floor(sumd[j][0]/sumd[j][1]) if bestM[j]>0 else np.ceil(sumd[j][0]/sumd[j][1]) for j in bestM}
    
    delta = {nodesL[i]:targets[clusters[i]]-degL[i] for i in range(len(clusters))}
    delta_min = [node for (node,defc) in delta.items() if defc<0]
    delta_plus = [node for (node,defc) in delta.items() if defc>0]

    Ganon = G.copy()
    failureCount = 0 ## ex: delta+ = [vi,vj], and (vi,vj) in E -> no edge addition is possible
    ## rn, just produces non anonymous graph
    
    ## we do random edge selection 
    while sum(delta.values()) < 0 and failureCount < 20: ## edge deletion
        try:
            vi,vj = random.sample(delta_min, k=2)
            vk = random.choice([vertex for vertex in Ganon.neighbors(vi) if vertex != vj])
            vl = random.choice([vertex for vertex in Ganon.neighbors(vj) if vertex != vi and vl not in Ganon.neighbors(vk)])
            Ganon.remove_edges_from([(vi,vk),(vj,vl)])
            Ganon.add_edge(vk,vl)
            delta[vi] += 1 ## - -1
            delta[vj] += 1
            delta_min = [node for (node,defc) in delta.items() if defc<0]
            failureCount=0
        except:
            failureCount += 1
        
    while sum(delta.values()) > 0 and failureCount < 20: ## edge addition
        try:
            vi,vj = random.sample(delta_plus, k=2)
            while (vi,vj) in Ganon.edges():
                vi,vj = random.sample(delta_plus, k=2)
            Ganon.add_edge(vi,vj)
            delta[vi] -= 1
            delta[vj] -= 1
            delta_plus = [node for (node,defc) in delta.items() if defc>0]    ## edge switch
            failureCount = 0
        except:
            failureCount += 1
    ## need sigma(dtilde) = sigma(d) = 0
    
    while sum(np.abs(list(delta.values()))) != 0 and failureCount <20: ## while there are deficits
        try:
            vi = random.choice(delta_min)
            vj = random.choice(delta_plus)
            vk = random.choice([v for v in Ganon.neighbors(vi) if v not in Ganon.neighbors(vj) and v != vj])
            Ganon.remove_edge(vi,vk)
            Ganon.add_edge(vj,vk)
            delta[vi] += 1
            delta[vj] -= 1
            delta_min = [node for (node,defc) in delta.items() if defc<0]
            delta_plus = [node for (node,defc) in delta.items() if defc>0]
            failureCount = 0
        except:
            failureCount += 1
    return Ganon

NameError: name 'nx' is not defined

In [ ]:
## Time testing -> v fast for n=1k, 0.02s, 

In [66]:
import time

In [73]:
BAG = nx.barabasi_albert_graph(10000, 500)

In [74]:
BAGanon = k_degSeq_Anon(BAG)

/var/folders/wk/bsc_wsqs3ns3xfy7kk02ss8m0000gn/T/ipykernel_29326/3154814199.py:20: RuntimeWarning: invalid value encountered in scalar divide
  probs = {j:(1- m[j][0]/(m[j][0]-m[j][1]),1 + m[j][1]/(m[j][0]-m[j][1])) for j in m}


init 4.656896114349365
112.75413012504578


it took 112 s for 10k nodes w/ h = 500 -> very manageble timeframe